In [18]:
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.util import ngrams
from collections import Counter
import numpy as np
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100000)

In [3]:
df = io.load('kof_features')

NameError: name 'io' is not defined

In [3]:
df['Retail_Price'] = df['Retail_Price'].astype(int)
df["Avg_Resale"] = df['Avg_Resale'].str.replace('[^\w\s]','')
df['Avg_Resale'] = df['Avg_Resale'].astype(int)

In [19]:
df

,Unnamed: 0,Code_Style,Name,Brand,Date,Retail_Price,Colorway,Story,KOF_Wants,Avg_Resale
0,0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air Foamposite One comes covered in a black and sail snakeskin textured material with black leather detailing on the eyestay that wraps all around the ankle. Additional black detailing can be seen on the tongue, laces, and inner liner while Habanero Red lands on the varios branding throughout the shoe, running down the tongue, on the pull tabs, and lastly on the rubber outsole.",2281,305
1,1,BV4580-400,Undercover x Nike SFB Mountain Obsidian University Red,Nike,2019-04-15,350,Obsidian/University Red-Dark Obsidian,"The Undercover x Nike SFB Mountain Obsidian/University Red is a collaboration from designer Jun Takahashi’s and Nike. Adding his own spin with an additional toggle lacing system and nylon straps, the silhouette is also detailed with “The New Warriors” branded across the straps. Finally, Undercover logos on the heels, co-branding on the insoles and its usual mesh upper and rugged outsole completes the design.",37,192
2,2,BV4580-001,Undercover x Nike SFB Mountain Black,Nike,2019-04-15,350,Black/Sail-Black,"The Undercover x Nike SFB Mountain Black is Introduced in a mixture of Black/Sail-Black. Undercover designer Jun Takahashi adds his twist to the design with an additional toggle lacing system and nylon straps. Another detail includes “The New Warriors” branded across the straps with Undercover logos on the heels, followed by co-branding on the insoles. Finally, its mesh upper and rugged outsole tops it all off.",70,231
3,3,AO2607-900,Nike PG 3 Mamba Mentality,Nike,2019-04-13,110,Multi-Color/Opti Yellow,"The Nike PG 3 Mamba Mentality comes covered in a black, yellow, grey, tan upper with a geometric pattern dominating a large portion of the shoe. Black overlays on the midfoot and heel keep the momentum going with large gold Swooshes landing on the lateral midsole. Paul George branding on the tongue and heels along with a yellow rubber outsole complete the look.",179,109
4,4,AO2918-102,Nike Kyrie 5 Mamba Mentality,Nike,2019-04-13,130,White/Cyber,"The Nike Kyrie 5 Mamba Mentality is a new rendition of Kyrie Irving’s signature silhouette that will be celebrating Mamba Day 2019. Covered in a color scheme of White and Cyber, the shoe sports neutral uppers with a hint of snakeskin detailing as a nod to Kobe Bryant’s alternate ego. Finally, hits of purple and neon green, along with red speckling on the heel and midsole are applied as a tribute to the 2009 Joker-inspired “Chaos” Zoom Kobe 5.\r\n\r\nGS pictured above.",271,107
5,5,CI1502-001,Nike Air Max 98 On Air La Mezcla,Nike,2019-04-13,200,White/Brown,"The Nike Air Max 98 “La Mezcla” is a new and exclusive iteration of the retro runner designed by Queens native and winner of NYC’s On Air competition from 2018, Gabrielle Serrano. Like the big city, the shoe sports a diverse mixture of hues with a gradient on the upper to represent the different race, ethnicity and cultural background of NYC. White fades to tan, light brown and black across the upper, while a textured mudguard, mismatching laces of light blue and lime green, along with matching swooshes and laces are detailed on the base. Finally, custom insoles, a white midsole and light blue Air Max cushioning completes the style.",723,177
6,6,CI1504-100,Nike Air Max 97 On Air London Summer of Love,Nike,2019-04-13,200,White/Green-Orange,"The Nike Air Max 97 London Summer of Love was designed by visual artist Jasmine Lasode. Sporting a vibrant color scheme of orange, green, blue, neon and purple hues across its upper, this iteration celebrates summer in the city and love, as Lasode used a personal memory of her time in London’s Primrose Hill as the source of her inspiration. Finally, the runner’s white mesh base acts as a canvas while finishing touches include “97P” printed in a pattern on the wavy

## Create "Purchase" feature


In [5]:
#create columns to calculate net profit
df['Price_Diff'] = df['Avg_Resale'] - df['Retail_Price']
df['Commission_Fee'] = abs((df['Avg_Resale']) * (9.5/100))
df['Seller_Fee'] = 5
df['Total_Credit'] = df['Price_Diff'] - df['Commission_Fee'] - df['Seller_Fee']
df['Cashout_Fee'] = abs((df['Total_Credit']) * (2.9/100))
df['Net_Profit'] = df['Total_Credit'] - df['Cashout_Fee']

In [6]:
#create purchase feature if the net profit is greater than 0
df['Purchase'] = np.where(df['Net_Profit'] > 0, 1, 0)

In [7]:
#drop columns used to calculate net profit
df.drop(['Price_Diff', 'Commission_Fee', 'Seller_Fee', 'Total_Credit', 'Cashout_Fee'], axis=1, inplace=True)

## Create "Brand_Code" feature

In [8]:
#identify all unique brands
unique_brands = list(df.Brand.unique())

#create to numerate each brand
brands = {}
counter = 0
for i in range(len(unique_brands)):
    current_brand = unique_brands[i]
    bcode = counter
    
    brands[current_brand] = bcode
    
    counter+=1

In [9]:
#create a list to generate brand codes for each row
BCode = []
for index, row in df.iterrows():
    bname = df.Brand[index]
    bcode = brands[bname]
    BCode.append(bcode)

In [10]:
#create new column brand code
df['Brand_Code'] = BCode

## Create "Color_Code" feature

In [11]:
print('There are ' + str(len(df)) + ' shoes in the database and ' + str(len(list(df.Colorway.unique()))) + ' unique colorways.')

There are 6338 shoes in the database and 4900 unique colorways.


In [12]:
#identify all unique brands
unique_colors = list(df.Colorway.unique())

#create to numerate each brand
colorways = {}
counter = 0
for i in range(len(unique_colors)):
    current_color = unique_colors[i]
    ccode = counter
    
    colorways[current_color] = ccode
    
    counter+=1

In [13]:
#create a list to generate brand codes for each row
CCode = []
for index, row in df.iterrows():
    cname = df.Colorway[index]
    ccode = colorways[cname]
    CCode.append(ccode)

In [14]:
#create new column brand code
df['Color_Code'] = CCode

## Create "Womens" feature

In [15]:
#create a list to identify if a shoe is a womens shoe or not (1=yes,0=no)
womens = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = 'wmns' in name
    checker2 = 'womens' in name
    checker3 = "women's" in name

    if checker1 == True or checker2 == True or checker3 == True:
        womens.append(1)
    else:
        womens.append(0)

In [16]:
df['Womens'] = womens

## Create "bCollab" feature

In [17]:
#create a list to identify if a shoe is a collaboartion or not (1=yes,0=no) -bcollab is boolean collaboration
bcollabs = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' x ' in name

    if checker1 == True:
        bcollabs.append(1)
    else:
        bcollabs.append(0)

In [18]:
df['bCollab'] = bcollabs

## Create "OG" feature

In [19]:
#create a list to identify if a shoe is OG or not (1=yes,0=no)
OG = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' og' in name

    if checker1 == True:
        OG.append(1)
    else:
        OG.append(0)

In [20]:
df['OG'] = OG

## Create "SP" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [21]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
SP = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' sp' in name

    if checker1 == True:
        SP.append(1)
    else:
        SP.append(0)

In [22]:
df['SP'] = SP

## Create "QS" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [23]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
QS = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' qs' in name

    if checker1 == True:
        QS.append(1)
    else:
        QS.append(0)

In [24]:
df['QS'] = QS

## Create "SB" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [25]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
SB = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' sb' in name

    if checker1 == True:
        SB.append(1)
    else:
        SB.append(0)

In [26]:
df['SB'] = SB

## Create "LS" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [27]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
LS = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' ls' in name

    if checker1 == True:
        LS.append(1)
    else:
        LS.append(0)

In [28]:
df['LS'] = LS

## Create "NRG" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [29]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
NRG = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' nrg' in name

    if checker1 == True:
        NRG.append(1)
    else:
        NRG.append(0)

In [30]:
df['NRG'] = NRG

## Create "PRM" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [31]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
PRM = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' prm' in name

    if checker1 == True:
        PRM.append(1)
    else:
        PRM.append(0)

In [32]:
df['PRM'] = PRM

## Create "NSW" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [33]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
NSW = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' nsw' in name

    if checker1 == True:
        NSW.append(1)
    else:
        NSW.append(0)

In [34]:
df['NSW'] = NSW

## Create "RETRO" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [35]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
RETRO = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' retro' in name

    if checker1 == True:
        RETRO.append(1)
    else:
        RETRO.append(0)

In [36]:
df['RETRO'] = RETRO

## Create "SE" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [37]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
SE = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' se' in name

    if checker1 == True:
        SE.append(1)
    else:
        SE.append(0)

In [38]:
df['SE'] = SE

## Create "PE" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [39]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
PE = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' pe' in name

    if checker1 == True:
        PE.append(1)
    else:
        PE.append(0)

In [40]:
df['PE'] = PE

## Create "GS" feature

Nike SP represents the highest level of Nike quality, and also an avenue for Nike’s creative endeavors. All NikeLab releases are SP and some collabs are also categorized as SP.

Example: Nike Free Flyknit Mercurial SP

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

In [41]:
#create a list to identify if a shoe is SP or not (1=yes,0=no)
GS = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' gs' in name

    if checker1 == True:
        GS.append(1)
    else:
        GS.append(0)

In [42]:
df['GS'] = GS

## Color Features

In [10]:
df['Story'].head()

0    Equipped with an eye-catching upper, this Air ...
1    The Undercover x Nike SFB Mountain Obsidian/Un...
2    The Undercover x Nike SFB Mountain Black is In...
3    The Nike PG 3 Mamba Mentality comes covered in...
4    The Nike Kyrie 5 Mamba Mentality is a new rend...
Name: Story, dtype: object

## Create "HS" feature

HS – Hyperstrike Shoes labeled HS are the most exclusive of all. They come in very limited quantities, with the majority given to friends and families (FNF) of artists and celebrities. 

Example: Nike Air Force 1 “Playstation”, which was a promotional shoe given to Sony employees back in 2006.

https://straatosphere.com/straatopedia-sneaker-terminology-guide/

### None were found in this data set. Was skipped for this iteration of the DB.

In [43]:
#create a list to identify if a shoe is HS or not (1=yes,0=no)
HS = []
for index, row in df.iterrows():
    name = df.Name[index]
    name = name.lower()
    
    checker1 = ' hs' in name

    if checker1 == True:
        HS.append(1)
    else:
        HS.append(0)

In [44]:
df['HS'] = HS

## FINAL_DF

In [52]:
df

,Unnamed: 0,Code_Style,Name,Brand,Date,Retail_Price,Colorway,Story,KOF_Wants,Avg_Resale,Net_Profit,Purchase,Brand_Code,Color_Code,Womens,bCollab,OG,SP,QS,SB,LS,NRG,PRM,NSW,RETRO,SE,PE,GS,HS
0,0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,49.545275,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,BV4580-400,Undercover x Nike SFB Mountain Obsidian Univer...,Nike,2019-04-15,350,Obsidian/University Red-Dark Obsidian,The Undercover x Nike SFB Mountain Obsidian/Un...,37,192,-186.495960,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,BV4580-001,Undercover x Nike SFB Mountain Black,Nike,2019-04-15,350,Black/Sail-Black,The Undercover x Nike SFB Mountain Black is In...,70,231,-150.177405,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,AO2607-900,Nike PG 3 Mamba Mentality,Nike,2019-04-13,110,Multi-Color/Opti Yellow,The Nike PG 3 Mamba Mentality comes covered in...,179,109,-16.829295,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,AO2918-102,Nike Kyrie 5 Mamba Mentality,Nike,2019-04-13,130,White/Cyber,The Nike Kyrie 5 Mamba Mentality is a new rend...,271,107,-39.271785,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,CI1502-001,Nike Air Max 98 On Air La Mezcla,Nike,2019-04-13,200,White/Brown,The Nike Air Max 98 “La Mezcla” is a new and e...,723,177,-46.114635,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6,CI1504-100,Nike Air Max 97 On Air London Summer of Love,Nike,2019-04-13,200,White/Green-Orange,The Nike Air Max 97 London Summer of Love was ...,2271,220,-6.071100,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7,CI1503-001,Nike Air Max 97 On Air Neon Seoul,Nike,2019-04-13,200,Black/Blue-Red,The Nike Air Max 97 Neon Seoul is a hybrid ite...,1103,239,10.967445,1,0,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,8,CI1505-001,Nike Air Max 1 On Air Tokyo Maze,Nike,2019-04-13,150,White/Black,"Designed by Yuta Takuman, this special colorwa...",472,205,29.639775,1,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9,CI1508-400,Nike Air Max 97 On Air Shanghai Kaleidoscope,Nike,2019-04-13,200,Blue/White-Red,"Designed by Cashu Ru, the Nike Air Max 97 Shan...",1233,496,236.807480,1,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
df.to_csv('KOF_04152019-modelv2.csv')